In [1]:
# Install required packages
!pip install -q transformers peft datasets torch accelerate huggingface_hub
!pip install -q scikit-learn


In [2]:
# =============================================================================
# COLAB SETUP CELL - Run this first!
# =============================================================================
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. This will be very slow!")

CUDA available: True
GPU: Tesla T4
GPU Memory: 15.8 GB


In [10]:
# =============================================================================
# OPTIMIZED LORA IMPLEMENTATION
# =============================================================================

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
# Corrected import for AdamW
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
from sklearn.metrics import accuracy_score
import time
from datasets import load_dataset
from tqdm.auto import tqdm
import gc


class OptimizedIMDBDataset(Dataset):
    """Memory-efficient dataset for Colab"""
    def __init__(self, texts, labels, tokenizer, max_length=256):  # Reduced max_length
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # Tokenize on-the-fly to save memory
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


def load_imdb_data_small():
    """Load smaller IMDB dataset for Colab"""
    print("Loading IMDB dataset (small subset for Colab)...")

    dataset = load_dataset("imdb")

    # Even smaller subsets for Colab free tier
    train_size = 25000  # Reduced from 5000
    test_size = 25000   # Reduced from 1000

    train_texts = dataset['train']['text'][:train_size]
    train_labels = dataset['train']['label'][:train_size]
    test_texts = dataset['test']['text'][:test_size]
    test_labels = dataset['test']['label'][:test_size]

    print(f"Training samples: {len(train_texts)}")
    print(f"Test samples: {len(test_texts)}")

    return train_texts, train_labels, test_texts, test_labels


def setup_colab_lora_model():
    """Setup LoRA model optimized for Colab"""
    print("Setting up LoRA model for Colab...")

    # Use DistilRoBERTa for faster training (smaller than RoBERTa)
    model_name = "distilroberta-base"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    base_model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    )

    print(f"Base model parameters: {sum(p.numel() for p in base_model.parameters()):,}")

    # Colab-optimized LoRA config
    lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,                        # Lower rank for speed
        lora_alpha=16,              # Correspondingly lower alpha
        lora_dropout=0.1,
        target_modules=['query', 'value']  # Focus on key attention components
    )

    # Apply LoRA
    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()

    # Clear some memory
    del base_model
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    return model, tokenizer


def train_epoch_colab(model, data_loader, optimizer, scheduler, device):
    """Memory-efficient training for Colab"""
    model.train()
    total_loss = 0
    progress_bar = tqdm(data_loader, desc="Training")

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        # Update progress bar
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})

        # Clear memory periodically
        if len(progress_bar) % 50 == 0:
            torch.cuda.empty_cache() if torch.cuda.is_available() else None

    return total_loss / len(data_loader)


def evaluate_model_colab(model, data_loader, device):
    """Memory-efficient evaluation for Colab"""
    model.eval()
    predictions = []
    actual_labels = []

    progress_bar = tqdm(data_loader, desc="Evaluating")

    with torch.no_grad():
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
            actual_labels.extend(labels.cpu().numpy())

    return accuracy_score(actual_labels, predictions)


def quick_comparison_demo(model):
    """Quick demo showing LoRA vs full fine-tuning comparison"""
    print("\n" + "="*50)
    print("PARAMETER COMPARISON DEMO")
    print("="*50)

    # DistilRoBERTa-base has approximately 82M parameters
    distilroberta_params = 82_000_000  # Approximate DistilRoBERTa parameters

    # Get actual LoRA parameters from the trained model
    lora_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    reduction = ((distilroberta_params - lora_params) / distilroberta_params) * 100

    print(f"Full Fine-tuning Parameters: {distilroberta_params:,}")
    print(f"LoRA Parameters: {lora_params:,}")
    print(f"Parameter Reduction: {reduction:.1f}%")
    print(f"Memory Savings: ~{reduction*0.8:.0f}%")  # Approximate memory savings

    return {
        'full_params': distilroberta_params,
        'lora_params': lora_params,
        'reduction': reduction
    }


def main():
    """Main function optimized for Colab execution"""
    print("🚀 Starting LoRA Fine-tuning on Google Colab!")
    print("="*60)

    # Device setup
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    if not torch.cuda.is_available():
        print("⚠️ WARNING: Running on CPU will be very slow!")
        print("Go to Runtime -> Change runtime type -> GPU")
        return

    # Load data
    train_texts, train_labels, test_texts, test_labels = load_imdb_data_small()

    # Setup model
    model, tokenizer = setup_colab_lora_model()
    model.to(device)

    # Create datasets with smaller batches for Colab
    train_dataset = OptimizedIMDBDataset(train_texts, train_labels, tokenizer)
    test_dataset = OptimizedIMDBDataset(test_texts, test_labels, tokenizer)

    # Smaller batch sizes for Colab memory limits
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # Bigger batch for eval

    # Optimizer setup (higher LR for LoRA)
    optimizer = AdamW(model.parameters(), lr=2e-4, eps=1e-8)
    epochs = 2  # Fewer epochs for Colab time limits
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    print(f"\nTraining Configuration:")
    print(f"- Epochs: {epochs}")
    print(f"- Batch size: 8")
    print(f"- Learning rate: 2e-4")
    print(f"- Total steps: {total_steps}")

    # Training loop
    print(f"\n🎯 Starting training on {len(train_texts)} samples...")
    start_time = time.time()

    for epoch in range(epochs):
        print(f"\n📚 Epoch {epoch + 1}/{epochs}")

        # Train
        train_loss = train_epoch_colab(model, train_loader, optimizer, scheduler, device)

        # Evaluate
        train_accuracy = evaluate_model_colab(model, train_loader, device)
        test_accuracy = evaluate_model_colab(model, test_loader, device)

        print(f"\n📊 Epoch {epoch + 1} Results:")
        print(f"   Train Loss: {train_loss:.4f}")
        print(f"   Train Accuracy: {train_accuracy:.4f}")
        print(f"   Test Accuracy: {test_accuracy:.4f}")

        # Memory cleanup
        gc.collect()
        torch.cuda.empty_cache() if torch.cuda.is_available() else None

    training_time = time.time() - start_time
    print(f"\n⏱️ Total training time: {training_time:.1f} seconds ({training_time/60:.1f} minutes)")

    # Show comparison
    comparison = quick_comparison_demo(model)

    # Final results
    print("\n" + "🎉 TRAINING COMPLETE!" + "🎉")
    print("="*60)
    print(f"Final Test Accuracy: {test_accuracy:.1%}")
    print(f"Parameter Reduction: {comparison['reduction']:.1f}%")
    print(f"Training Time: {training_time/60:.1f} minutes")
    print("\n💡 Key Takeaways:")
    print("   • LoRA achieves great performance with 99%+ fewer parameters")
    print("   • Perfect for fine-tuning on limited hardware like Colab")
    print("   • Much faster than full fine-tuning")

    return model, tokenizer, test_accuracy

In [11]:
# =============================================================================
# COLAB EXECUTION CELL - Run this to start training!
# =============================================================================

if __name__ == "__main__":
    # Run the optimized training
    model, tokenizer, accuracy = main() # Changed colab_main to main

    print(f"\n✅ Training completed! Final accuracy: {accuracy:.1%}")

🚀 Starting LoRA Fine-tuning on Google Colab!
Using device: cuda
Loading IMDB dataset (small subset for Colab)...
Training samples: 25000
Test samples: 25000
Setting up LoRA model for Colab...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Base model parameters: 82,119,938
trainable params: 739,586 || all params: 82,859,524 || trainable%: 0.8926

Training Configuration:
- Epochs: 2
- Batch size: 8
- Learning rate: 2e-4
- Total steps: 6250

🎯 Starting training on 25000 samples...

📚 Epoch 1/2


Training:   0%|          | 0/3125 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3125 [00:00<?, ?it/s]

...


📊 Epoch 1 Results:
   Train Loss: 0.2963
   Train Accuracy: 0.9220
   Test Accuracy: 0.9175

📚 Epoch 2/2


Training:   0%|          | 0/3125 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/3125 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1563 [00:00<?, ?it/s]


📊 Epoch 2 Results:
   Train Loss: 0.2470
   Train Accuracy: 0.9268
   Test Accuracy: 0.9205

⏱️ Total training time: 1662.1 seconds (27.7 minutes)

PARAMETER COMPARISON DEMO
Full Fine-tuning Parameters: 82,000,000
LoRA Parameters: 739,586
Parameter Reduction: 99.1%
Memory Savings: ~79%

🎉 TRAINING COMPLETE!🎉
Final Test Accuracy: 92.1%
Parameter Reduction: 99.1%
Training Time: 27.7 minutes

💡 Key Takeaways:
   • LoRA achieves great performance with 99%+ fewer parameters
   • Perfect for fine-tuning on limited hardware like Colab
   • Much faster than full fine-tuning

✅ Training completed! Final accuracy: 92.1%
